<a href="https://colab.research.google.com/github/rohitd2209/TFX/blob/main/Well_state_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
!pip install -U tfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.8.2
TFX version: 1.8.0


In [2]:
import os
#pipeline for schema generation
PIPELINE_NAME = "well-simple"
SCHEMA_PIPELINE_NAME = "well-tfdv-schema"

# Output directory to store artifacts generated from the pipeline
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')

# Output directory where created models from the pipeline will be exported
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level

In [4]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='well_tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/rohitd2209/TFX/main/Well_state_prediction.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('/tmp/well_tfx-datafsg9u0qg/data.csv',
 <http.client.HTTPMessage at 0x7f98f755b390>)

In [5]:
!head {_data_filepath}

tempC,WindChillC,winddirDegree,WindGustKmph,windspeedKmph,wellState
111.00,121.00,131.00,141.00,151.00,Tenth
57.00,53.00,59.00,57.00,59.00,Fifth
14.00,16.00,13.00,17.00,19.00,First
22.00,26.00,27.00,28.00,25.00,Second
132.00,143.00,176.00,165.00,187.00,Tenth
14.00,16.00,13.00,17.00,18.00,First
27.00,24.00,28.00,23.00,26.00,Second
94.00,92.00,93.00,91.00,90.00,Nineth
112.00,122.00,132.00,142.00,152.00,Tenth


In [6]:

!sed -i '/\bNA\b/d' {_data_filepath}
!head {_data_filepath}

tempC,WindChillC,winddirDegree,WindGustKmph,windspeedKmph,wellState
111.00,121.00,131.00,141.00,151.00,Tenth
57.00,53.00,59.00,57.00,59.00,Fifth
14.00,16.00,13.00,17.00,19.00,First
22.00,26.00,27.00,28.00,25.00,Second
132.00,143.00,176.00,165.00,187.00,Tenth
14.00,16.00,13.00,17.00,18.00,First
27.00,24.00,28.00,23.00,26.00,Second
94.00,92.00,93.00,91.00,90.00,Nineth
112.00,122.00,132.00,142.00,152.00,Tenth


In [7]:
def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # ExampleGen
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # StatisticsGen
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # SchemaGen
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [8]:
tfx.orchestration.LocalDagRunner().run(
  _create_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=SCHEMA_METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "CsvExampleGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.example_gen.csv_example_gen.executor.Executor"
      }
    }
  }
}
executor_specs {
  key: "SchemaGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.schema_gen.executor.Executor"
    }
  }
}
executor_specs {
  key: "StatisticsGen"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "tfx.components.statistics_gen.executor.Executor"
      }
    }
  }
}
custom_driver_specs {
  key: "CsvExampleGen"
  value {
    python_class_executable_spec {
      class_path: "tfx.components.example_gen.driver.FileBasedDriver"
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "metada

INFO:absl:Processing input csv data /tmp/well_tfx-datafsg9u0qg/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Value type <class 'NoneType'> of key version in exec_properties is not supported, going to drop it
INFO:absl:Value type <class 'list'> of key _beam_pipeline_args in exec_properties is not supported, going to drop it
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 4 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'examples': [Artifact(artifact: uri: "pipelines/well-tfdv-schema/CsvExampleGen/examples/4"
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:1228,xor_checksum:1655096296,sum_checksum:1655096296"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "well-tfdv-schema:2022-06-13T04:58:20.070891:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0


In [9]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_artifacts_dict(metadata, latest_execution.id,
                                          [metadata_store_pb2.Event.OUTPUT])

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

In [10]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

INFO:absl:MetadataStore with DB connection initialized


In [11]:
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

In [12]:
visualize_artifacts(schema_artifacts)

,Type,Presence,Valency,Domain
Feature name,,,,
'WindChillC',FLOAT,required,,-
'WindGustKmph',FLOAT,required,,-
'tempC',FLOAT,required,,-
'wellState',STRING,required,,'wellState'
'winddirDegree',FLOAT,required,,-
'windspeedKmph',FLOAT,required,,-


,Values
Domain,
'wellState',"'Fifth', 'First', 'Fourth', 'Nineth', 'Second', 'Seventh', 'Tenth', 'Sixth'"


In [13]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

'schema/schema.pbtxt'

In [14]:
print(f'Schema at {SCHEMA_PATH}-----')
!cat {SCHEMA_PATH}/*

Schema at schema-----
feature {
  name: "WindChillC"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "WindGustKmph"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "tempC"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "wellState"
  type: BYTES
  domain: "wellState"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "winddirDegree"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "windspeedKmph"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
string_domain {
  name: "wellState"
  value: "Fifth"
  value: "First"
  

In [15]:
_module_file = 'well_trainer.py'

In [24]:
%%writefile {_module_file}


from typing import List, Text
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio


#      Transfrom component  
_FEATURE_KEYS = [
      'tempC','WindChillC','winddirDegree','WindGustKmph','windspeedKmph'

]
_LABEL_KEY = 'wellState'

_TRAIN_BATCH_SIZE = 10
_EVAL_BATCH_SIZE = 5


# NEW: TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature.
  """
  outputs = {}

  # Uses features defined in _FEATURE_KEYS only.
  for key in _FEATURE_KEYS:
    # tft.scale_to_z_score computes the mean and variance of the given feature
    # and scales the output based on the result.
    outputs[key] = tft.scale_to_z_score(inputs[key])

  # For the label column we provide the mapping from string to index.
  # We could instead use `tft.compute_and_apply_vocabulary()` in order to
  # compute the vocabulary dynamically and perform a lookup.
  # Since in this example there are only 2 possible values, we use a hard-coded
  # table for simplicity.
  table_keys = ['Fifth', 'First', 'Fourth', 'Nineth', 'Second', 'Seventh', 'Sixth', 'Tenth']
  initializer = tf.lookup.KeyValueTensorInitializer(
      keys=table_keys,
      values=tf.cast(tf.range(len(table_keys)), tf.float32),
      key_dtype=tf.string,
      value_dtype=tf.float32)
  table = tf.lookup.StaticHashTable(initializer, default_value=-1)
  outputs[_LABEL_KEY] = table.lookup(inputs[_LABEL_KEY])

  return outputs


# NEW: This function will apply the same transform operation to training data
#      and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  transformed_features = tft_layer(raw_features)
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  else:
    return transformed_features, None


# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format.
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema includes unnecessary fields like 'species' and
    # 'island', we filter feature_spec to include required keys only.
    required_feature_spec = {
        k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS
    }
    parsed_features = tf.io.parse_example(serialized_tf_examples,
                                          required_feature_spec)

    # Preprocess parsed input with transform operation defined in
    # preprocessing_fn().
    transformed_features, _ = _apply_preprocessing(parsed_features,
                                                   model.tft_layer)
    # Run inference with ML model.
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 10) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
      schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying spaceship data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=key)
      for key in _FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(8)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalCrossentropy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # NEW: Save a computation graph including transform layer.
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting well_trainer.py


In [25]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Implements the spaceship pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')


########       ExampleValidator                               ##########
  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])


##########       Transform             #################
  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],

      # NEW: Pass transform_graph to the trainer.
      transform_graph=transform.outputs['transform_graph'],

      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator,

      transform,  # NEW: Transform component was added to the pipeline.

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

In [26]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Generating ephemeral wheel package for '/content/well_trainer.py' (including modules: ['well_trainer']).
INFO:absl:User module package has hash fingerprint version a555a9315ceb55f856d37f8bae94bd8354e4b367ef2081761d6abd3d0d6e4b9a.
INFO:absl:Executing: ['/usr/bin/python3', '/tmp/tmp6kp5hxzy/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmp2ejv1w6y', '--dist-dir', '/tmp/tmp4bh6p63g']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/well-simple/_wheels/tfx_user_code_Transform-0.0+a555a9315ceb55f856d37f8bae94bd8354e4b367ef2081761d6abd3d0d6e4b9a-py3-none-any.whl'; target user module is 'well_trainer'.
INFO:absl:Full user module path is 'well_trainer@pipelines/well-simple/_wheels/tfx_user_code_Transform-0.0+a555a9315ceb55f856d37f8bae94bd8354e4b367ef2081761d6abd3d0d6e4b9a-py3-none-any.whl'
INFO:absl:Generating ephemer

INFO:tensorflow:Assets written to: pipelines/well-simple/Transform/transform_graph/40/.temp_path/tftransform_tmp/0c6ccaa40ad54f3a8dd6d3562d88f1b6/assets


INFO:tensorflow:Assets written to: pipelines/well-simple/Transform/transform_graph/40/.temp_path/tftransform_tmp/0c6ccaa40ad54f3a8dd6d3562d88f1b6/assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Feature WindChillC has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature WindGustKmph has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature tempC has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature wellState has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature winddirDegree has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature windspeedKmph has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Model: "model_2"
INFO:absl:__________________________________________________________________________________________________
INFO:absl: Layer (type)                   Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl: tempC (InputLayer)             [(None, 1)]          0           []       

100/100 [==============================] - 1s 5ms/step - loss: 1.1219 - sparse_categorical_crossentropy: 2.3067 - val_loss: 1.2548 - val_sparse_categorical_crossentropy: 1.2566
INFO:tensorflow:Assets written to: pipelines/well-simple/Trainer/model/42/Format-Serving/assets


INFO:tensorflow:Assets written to: pipelines/well-simple/Trainer/model/42/Format-Serving/assets
INFO:absl:Training complete. Model written to pipelines/well-simple/Trainer/model/42/Format-Serving. ModelRun written to pipelines/well-simple/Trainer/model_run/42
INFO:absl:Cleaning up stateless execution info.
INFO:absl:Execution 42 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'model_run': [Artifact(artifact: uri: "pipelines/well-simple/Trainer/model_run/42"
custom_properties {
  key: "name"
  value {
    string_value: "well-simple:2022-06-13T05:09:13.871567:Trainer:model_run:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.8.0"
  }
}
name: "well-simple:2022-06-13T05:09:13.871567:Trainer:model_run:0"
, artifact_type: name: "ModelRun"
)], 'model': [Artifact(artifact: uri: "pipelines/well-simple/Trainer/model/42"
custom_properties {
  key: "name"
  value {
    string_value: "well-s

In [27]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

serving_model/well-simple
serving_model/well-simple/1655096994
serving_model/well-simple/1655096994/assets
serving_model/well-simple/1655096994/saved_model.pb
serving_model/well-simple/1655096994/keras_metadata.pb
serving_model/well-simple/1655096994/variables
serving_model/well-simple/1655096994/variables/variables.index
serving_model/well-simple/1655096994/variables/variables.data-00000-of-00001


In [28]:
!saved_model_cli show --dir {SERVING_MODEL_DIR}/$(ls -1 {SERVING_MODEL_DIR} | sort -nr | head -1) --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['examples'] tensor_info:
      dtype: DT_STRING
      shape: (-1)
      name: serving_default_examples:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_0'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 8)
      name: StatefulPartitionedCall_2:0
Method name is: tensorflow/serving/predict


In [29]:
# Find a model with the latest timestamp.
model_dirs = (item for item in os.scandir(SERVING_MODEL_DIR) if item.is_dir())
model_path = max(model_dirs, key=lambda i: int(i.name)).path

loaded_model = tf.keras.models.load_model(model_path)
inference_fn = loaded_model.signatures['serving_default']

In [30]:
# Prepare an example and run inference.
features = {

'tempC' : tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
'WindChillC' : tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
'winddirDegree' : tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
'WindGustKmph' : tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
'windspeedKmph' : tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),


}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())



[[ 0.86979103  0.8853868   1.0757654  -2.695374    1.3008902  -2.8055649
  -3.5817356  -2.452659  ]]


In [30]:
['Fifth', 'First', 'Fourth', 'Nineth', 'Second', 'Seventh', 'Sixth', 'Tenth']